In [5]:
# imports
from importlib import reload
import numpy as np
import pandas
import datetime
import sqlalchemy

from mhw_analysis.events import utils as mhwe_utils

# Testing 123..

## mhw_analysis

In [2]:
engine = mhwe_utils.db_engine()
connection = engine.connect()

In [7]:
reload(mhwe_utils)
mhw_events = mhwe_utils.query_db(engine, latminx=(30., 40.), lonminx=(170., 180.))
mhw_events.head()

,date,lat,lon,ievent,duration,category,time_start
0,1990-01-31,30.125,170.125,9,8,0,726498
1,1990-03-04,30.125,170.125,10,7,0,726530
2,1990-08-20,30.125,170.125,11,6,0,726699
3,1990-01-26,30.375,170.125,14,12,0,726493
4,1990-08-20,30.375,170.125,15,6,1,726699


## Laborious

In [3]:
# Connect
mhw_file = '/home/xavier/Projects/Oceanography/MHW/db/mhws_allsky_defaults.db'
engine = sqlalchemy.create_engine('sqlite:///' + mhw_file)
connection = engine.connect()

In [3]:
metadata = sqlalchemy.MetaData()
metadata

MetaData(bind=None)

In [4]:
mhw_tbl = sqlalchemy.Table('MHW_Events', metadata, autoload=True, autoload_with=engine)

In [5]:
mhw_tbl

Table('MHW_Events', MetaData(bind=None), Column('index', BIGINT(), table=<MHW_Events>), Column('time_start', INTEGER(), table=<MHW_Events>), Column('time_end', INTEGER(), table=<MHW_Events>), Column('time_peak', INTEGER(), table=<MHW_Events>), Column('duration', INTEGER(), table=<MHW_Events>), Column('duration_moderate', INTEGER(), table=<MHW_Events>), Column('duration_strong', INTEGER(), table=<MHW_Events>), Column('duration_severe', INTEGER(), table=<MHW_Events>), Column('duration_extreme', INTEGER(), table=<MHW_Events>), Column('category', INTEGER(), table=<MHW_Events>), Column('n_events', INTEGER(), table=<MHW_Events>), Column('ievent', BIGINT(), table=<MHW_Events>), Column('date', DATETIME(), table=<MHW_Events>), Column('lat', FLOAT(), table=<MHW_Events>), Column('lon', FLOAT(), table=<MHW_Events>), Column('intensity_max', FLOAT(), table=<MHW_Events>), Column('intensity_mean', FLOAT(), table=<MHW_Events>), Column('intensity_var', FLOAT(), table=<MHW_Events>), Column('intensity_cum

## Query

In [6]:
mhw_tbl.columns['date']

Column('date', DATETIME(), table=<MHW_Events>)

In [7]:
mhw_tbl.columns.date

Column('date', DATETIME(), table=<MHW_Events>)

In [8]:
mhw_tbl.columns.ievent >= 0

In [6]:
query = sqlalchemy.select([mhw_tbl.columns.date, mhw_tbl.columns.lat, mhw_tbl.columns.lon, mhw_tbl.columns.ievent,
                          mhw_tbl.columns.duration, mhw_tbl.columns.time_start, mhw_tbl.columns.category,
                          ]).where(
    mhw_tbl.columns.ievent >= 0)

In [14]:
datetime.datetime.strptime('DEC 01 2019', '%b %d %Y')

datetime.datetime(2019, 12, 1, 0, 0)

In [17]:
tmp = (2019, 12, 1)

In [19]:
datetime.datetime(*tmp)

datetime.datetime(2019, 12, 1, 0, 0)

## Execute

In [7]:
result = connection.execute(query)

### Partial

In [58]:
partial_results = pandas.DataFrame(result.fetchmany(50), columns=['date', 'lat', 'lon', 'ievent',
                                                                 'duration', 'time_start', 'category'])

In [59]:
partial_results.head()

,date,lat,lon,ievent,duration,time_start,category
0,1992-04-24,-70.375,0.125,28,6,727312,0
1,1992-05-22,-70.375,0.125,29,37,727340,0
2,1992-07-14,-70.375,0.125,30,33,727393,0
3,1992-09-03,-70.375,0.125,31,7,727444,0
4,1992-09-18,-70.375,0.125,32,95,727459,3


### All

In [8]:
%time results = pandas.DataFrame(result.fetchall(),
                                columns=['date', 'lat', 'lon', 'ievent',
                                        'duration', 'time_start', 'category'])

CPU times: user 4min 22s, sys: 18.3 s, total: 4min 40s
Wall time: 4min 40s


In [9]:
results.head()

,0,1,2,3,4,5,6
0,1982-01-01,-71.375,0.125,0,13,723546,2
1,1982-12-20,-71.375,0.125,1,33,723899,2
2,1985-01-30,-71.375,0.125,2,10,724671,0
3,1988-01-10,-71.375,0.125,3,9,725746,0
4,1991-01-09,-71.375,0.125,4,52,726841,0


In [10]:
len(results)

57685830

In [26]:
results.columns

RangeIndex(start=0, stop=2, step=1)

----

# Full Table

In [6]:
mhw_events = pandas.read_sql_table('MHW_Events', con=engine) #, columns=['date', 'lon', 'lat', 'duration', 'ievent', 'time_start', 'index', 'category'])
#mhw_events.set_index('date')

In [7]:
len(mhw_events)

57685830

In [8]:
np.min(mhw_events['lat'])

-78.375

In [9]:
mhw_events.head()

,index,time_start,time_end,time_peak,duration,duration_moderate,duration_strong,duration_severe,duration_extreme,category,...,intensity_max_relThresh,intensity_max_abs,intensity_mean_relThresh,intensity_mean_abs,intensity_var_relThresh,intensity_var_abs,intensity_cumulative_relThresh,intensity_cumulative_abs,rate_onset,rate_decline
0,0,723546,723558,723552,13,1,7,5,0,2,...,0.649290,-0.20,0.466228,-0.383077,0.135821,0.120153,6.060967,-4.980000,0.026651,0.091177
1,1,723899,723931,723901,33,14,18,1,0,2,...,0.590032,-0.36,0.316166,-0.546061,0.150839,0.145121,10.433483,-18.019997,0.281878,0.020300
2,2,724671,724680,724677,10,10,0,0,0,0,...,0.073226,-0.70,0.046994,-0.724000,0.022049,0.022891,0.469935,-7.239999,0.010766,0.019172
3,3,725746,725754,725752,9,9,0,0,0,0,...,0.082419,-0.71,0.046864,-0.758889,0.022717,0.033811,0.421774,-6.830000,0.014714,0.027270
4,4,726841,726892,726858,52,52,0,0,0,0,...,0.252129,-0.51,0.101606,-0.698269,0.070642,0.099087,5.283517,-36.310009,0.015743,0.006539


In [17]:
np.min(mhw_events.ievent)

0

## Write to HDF

In [10]:
mhw_events.to_hdf('mhw_events.hdf', 'MHW_Events', mode='w')